In [4]:
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

In [5]:
torch.manual_seed(10)

# intro
机器学习的算法具有非常通用的编程范式(Paradigm)，

1. data
2. model
3. target 
4. algorithm
5. visualization

因此，对于不同的算法，我们只需要在范式内修改即可，

因此，机器学习的算法实现并不具有技巧性，难点在于数学层面的推导和理解

下面给出机器学习算法的范式，在后面我们还会针对数据的加载和可视化两个部分进行优化
1. 如何加载数据，并以tensor的形式输入到模型？
2. 如何动态可视化训练过程的loss，acc等？

# 1. 数据

In [6]:
sample_nums = 100
mean_value = 1.7
bias = 1
n_data = torch.ones(sample_nums, 2) 

# use normal distribution to generate training samples
x0 = torch.normal(mean_value *  n_data, 1) + bias # 在(1, 1.7)处生成normal分布的数据
y0 = torch.zeros(sample_nums) # labeled as 0

x1 = torch.normal(-mean_value * n_data, 1) + bias
y1 = torch.ones(sample_nums)

train_x = torch.cat((x0, x1), 0) # T x 2 
train_y = torch.cat((y0, y1), 0) # T x 1

# 2. 模型
模型就是一个从输入到输出的带参映射

这里nn.Linear的参数由weights和bias组成，其中weights和bias的ndim都是1

In [7]:
class LR(nn.Module):
    def __init__(self):
        """
            在__init__中描述模型的building blocks有哪些，所有的building blocks应该在这里引入
        """
        super(LR, self).__init__()
        self.features = nn.Linear(2, 1) # 2 -> 1, 1x2矩阵，包含两个参数，分别是weight和bias
        self.sigmoid = nn.Sigmoid() 
    
    def forward(self, x):
        """
            在forward中描述模型的计算过程，即如何从输入得到输出
        """
        x = self.features(x)
        x = self.sigmoid(x)
        return x

lr_net = LR() # Tx2 -(Linear) -> Tx1 -(Sigmoid) -> Tx1 

# 3. 目标/损失函数

In [8]:
loss_fn = nn.BCELoss()

# 4. 算法 & 可视化过程
值得注意的是这里计算acc的方式比较tricky:
```python
mask = y_pred.ge(0.5).float().squeeze()
correct = (mask == train_y).sum()
acc = correct.item() / train_y.size()
```

In [2]:
lr = 0.01
optimizer = torch.optim.SGD(lr_net.parameters(), lr=lr, momentum=0.9)

for iteration in range(1000):
    # forward: y_pred = model(x)
    y_pred = lr_net(train_x)
    # loss
    loss = loss_fn(y_pred.squeeze(), train_y) # y_pred包含创建它的函数信息，因此能向前追溯，进行反向传播
    # backpropagation
    loss.backward()
    # update parameter
    optimizer.step()
    # clear old grads
    optimizer.zero_grad()
    # drawing 
    if iteration % 20 == 0:
        mask = y_pred.ge(0.5).float().squeeze()
        correct = (mask == train_y).sum()
        acc = correct.item() / train_y.size(0)
        
        plt.scatter(x0.data.numpy()[:, 0], x0.data.numpy()[:, 1], c='r', label='class 0')
        plt.scatter(x1.data.numpy()[:, 0], x1.data.numpy()[:, 1], c='b', label='class 1')
        
        w0, w1 = lr_net.features.weight[0]
        w0, w1 = float(w0.item()), float(w1.item())
        plot_b = float(lr_net.features.bias[0].item())
        plot_x = np.arange(-6, 6, 0.1)
        plot_y = (-w0 * plot_x - plot_b) / w1
        
        plt.xlim(-5, 7)
        plt.ylim(-7, 7)
        plt.plot(plot_x, plot_y)
        
        plt.text(-5, 5, 'Loss=%.4f' % loss.data.numpy(), fontdict={'size': 20, 'color': 'red'})
        plt.title("Iteration: {}\nw0:{:.2f} w1:{:.2f} b: {:.2f} accuracy:{:.2%}".format(iteration, w0, w1, plot_b, acc))
        plt.legend()
        # plt.savefig(str(iteration / 20)+".png")
        plt.show()
        plt.pause(0.5)
        # 如果准确率大于 99%，则停止训练
        if acc > 0.99:
            break

NameError: name 'torch' is not defined